In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19191221/tmpz0uqhkc0
  JVM stdout: /state/partition1/job-19191221/tmpz0uqhkc0/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19191221/tmpz0uqhkc0/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_mk7516_m189t0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.542 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(40975)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(1296, 7)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_1_20220507_121119_model_256


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,171.0,684.0,291167.0,5.0,9.0,7.27193,6.0,34.0,29.127193




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 1.3114287571792107e-06
RMSE: 0.001145176299605965
LogLoss: 0.0003507354835287154
Mean Per-Class Error: 0.0
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,Error,Rate
0,916.0,0.0,0.0,0.0,0.0,0 / 916
1,0.0,281.0,0.0,0.0,0.0,0 / 281
2,0.0,0.0,45.0,0.0,0.0,0 / 45
3,0.0,0.0,0.0,54.0,0.0,0 / 54
4,916.0,281.0,45.0,54.0,0.0,"0 / 1,296"



Top-4 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.993053,0.003234,0.996154,0.988417,0.992278,0.992278,0.996139
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.006947,0.003234,0.003846,0.011583,0.007722,0.007722,0.003861
3,err_count,1.800000,0.836660,1.000000,3.000000,2.000000,2.000000,1.000000
4,logloss,0.027309,0.011338,0.042466,0.031748,0.027001,0.011393,0.023937
5,max_per_class_error,0.039896,0.031359,0.017544,0.016043,0.076923,0.071429,0.017544
6,mean_per_class_accuracy,0.988336,0.010142,0.995614,0.995989,0.976863,0.977597,0.995614
7,mean_per_class_error,0.011664,0.010142,0.004386,0.004011,0.023137,0.022403,0.004386
8,mse,0.006341,0.002190,0.006870,0.009394,0.006624,0.003398,0.005420
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-05-07 12:53:58,13 min 45.532 sec,0.0,0.750000,1.386294,0.444444,NaN,NaN
1,,2022-05-07 12:53:58,13 min 45.549 sec,5.0,0.488449,0.672888,0.066358,NaN,NaN
2,,2022-05-07 12:53:58,13 min 45.564 sec,10.0,0.340826,0.397937,0.057099,NaN,NaN
3,,2022-05-07 12:53:58,13 min 45.581 sec,15.0,0.262976,0.262985,0.040123,NaN,NaN
4,,2022-05-07 12:53:58,13 min 45.598 sec,20.0,0.214329,0.185523,0.019290,NaN,NaN
5,,2022-05-07 12:53:58,13 min 45.613 sec,25.0,0.178965,0.136900,0.015432,NaN,NaN
6,,2022-05-07 12:53:58,13 min 45.631 sec,30.0,0.150659,0.104319,0.005401,NaN,NaN
7,,2022-05-07 12:53:58,13 min 45.647 sec,35.0,0.129418,0.082854,0.003858,NaN,NaN
8,,2022-05-07 12:53:58,13 min 45.664 sec,40.0,0.110874,0.066349,0.002315,NaN,NaN
9,,2022-05-07 12:53:58,13 min 45.680 sec,45.0,0.090856,0.050656,0.001543,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,safety,657.853271,1.000000,0.307500
1,persons,578.589111,0.879511,0.270449
2,buying,326.696686,0.496610,0.152708
3,maint,325.530670,0.494838,0.152162
4,lug_boot,185.706512,0.282292,0.086805
5,doors,64.986061,0.098785,0.030376


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
GBM_grid_1_AutoML_1_20220507_121119_model_256,0.00989681,0.0266368,0.0796402,0.00634257
GBM_grid_1_AutoML_1_20220507_121119_model_317,0.0122012,0.0219199,0.0706737,0.00499478
GBM_grid_1_AutoML_1_20220507_121119_model_249,0.0122583,0.0502782,0.108791,0.0118354
GBM_grid_1_AutoML_1_20220507_121119_model_180,0.0127471,0.0357431,0.0893558,0.00798447
GBM_grid_1_AutoML_1_20220507_121119_model_13,0.0128041,0.0568426,0.114876,0.0131966
GBM_grid_1_AutoML_1_20220507_121119_model_292,0.0130562,0.0288924,0.0812366,0.00659938
GBM_grid_1_AutoML_1_20220507_121119_model_191,0.0134001,0.024233,0.0728171,0.00530234
GBM_grid_1_AutoML_1_20220507_121119_model_189,0.013673,0.0259844,0.0769218,0.00591697
GBM_grid_1_AutoML_1_20220507_121119_model_207,0.0139459,0.0295299,0.08412,0.00707618
GBM_grid_1_AutoML_1_20220507_121119_model_267,0.0139459,0.0317327,0.0849821,0.00722196
